<a href="https://colab.research.google.com/github/guilhermemrv/portfolio/blob/master/04_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nesse projeto faremos scraping de um site e em seguida retornaremos os dados em um dataframe.

Habilidades desenvolvidas no código abaixo:

1. Realização de Scraping usando o BeautifulSoup, utilizando as tags e classes HTML para identificação das informações desejadas.
2. Criação de loop para extrair informações de uma sequência de websites.
3. Criação de um DataFrame com as informações obtidas.

Você pode acessar o site para entender os dados que serão extraidos:
https://alura-site-scraping.herokuapp.com/index.php

In [ ]:
import bs4
import urllib.request as urllib_request
import pandas as pd

from urllib.request import urlretrieve, urlopen
from bs4 import BeautifulSoup

#O site utilizado foi criado para fins educativos.
url = 'https://alura-site-scraping.herokuapp.com/index.php'
response = urlopen(url)
html = response.read().decode('utf-8')
#obtemos o código inteiro da página
#decode converte a pagina de bytes para string


In [ ]:
#Vamos tratar o codigo e retirar os espaços em branco
#Para isso criaremos uma função, que posteriormente será repetido.
#split irá separar cada item do código, o join e o replace irão retirar os espaços.
def trata_html(input):
    return " ".join(input.split()).replace('> <','><')

html = trata_html(html)

In [ ]:
#Para acessar as informações usaremos o Beautiful Soup

soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#A partir de agora, iremos analisar a página que iremos fazer o scraping e tiraremos as informações que são relevantes para automatizar a retirada de dados
#Iniciaremos identificando quantas páginas serão abertas e retirada
#Verificamos que tal informação se encontra na tag 'span'. Atenção para colocar um underline após a class, pois o python pode confundir o comando.

texto_paginas = soup.find('span',class_="info-pages").get_text()
texto_paginas

'Página 1 de 25'

In [ ]:
#No caso iremos separar cada palavra da frase e pegar o último. 
pages = int(soup.find('span',class_="info-pages").get_text().split()[-1])
pages

25

Iremos fazer agora um loop para abrir cada página e recolher as informações que queremos de cada ítem

Note que os dados estão divididos em cards, e utilizaremos essa estrutura


In [ ]:
#Criamos a lista que será adicionado os dicionários
cards = []

for i in range(pages):
   
    #Identificamos que cada página difere apenas no último caractere, então ele que vai ser o primeiro loop
    response = urlopen('https://alura-site-scraping.herokuapp.com/index.php?page=' + str(i +1) )
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    #'anuncios' vai buscar as informações de cada card
    anuncios = soup.find('div',{'id':'container-cards'}).findAll('div',class_="card") 
    
    #Vamos fazer o segundo loop, buscando cada card para cada página
    for anuncio in anuncios:
        #Criamos um dicionario
        card = {}

        #adicionamos o ítem 'value' ao dicionário
        card['value'] = anuncio.find('p',{'class':'txt-value'}).getText()

        infos = anuncio.find('div', {'class':'body-card'}).findAll('p')

        #mais um loop, agora para cada informação dentro do card
        for info in infos:
            #adicionamos as informações ao dicionario
            card[info.get('class')[0].split('-')[-1]]=info.get_text()

        items = anuncio.find('div',{'class':'body-card'}).ul.findAll('li')
        items.pop()
        acessorios = []
        for item in items: 
            acessorios.append(item.getText().replace('► ', ''))
        
        #adicionar agora os acessorios no dicionario
        card['items'] = acessorios

        cards.append(card)

In [ ]:
#Criando um dataframe com os items extraídos
dataset = pd.DataFrame(cards)

In [ ]:
dataset.sample(5)

,value,name,category,motor,description,location,items,opportunity
140,R$ 422.000,CUBA,USADO,Motor Diesel,Ano 1992 - 62.255 km,Rio de Janeiro - RJ,"[Vidros elétricos, Controle de tração, Piloto ...",NaN
89,R$ 201.000,OLDTIMERS,USADO,Motor Diesel,Ano 2015 - 41.735 km,Belo Horizonte - MG,"[Vidros elétricos, Ar condicionado, Controle d...",NaN
202,R$ 318.000,PORSCHE TARGA,USADO,Motor 1.8 16v,Ano 2005 - 102.500 km,Belo Horizonte - MG,"[Teto panorâmico, Câmbio automático, Sensor cr...",NaN
214,R$ 155.000,VOLKSWAGEN,USADO,Motor Diesel V8,Ano 2017 - 64.852 km,Belo Horizonte - MG,"[Câmera de estacionamento, Central multimídia,...",OPORTUNIDADE
231,R$ 496.000,CORVETTE YELLOW,NOVO,Motor 5.0 V8 Bi-Turbo,Ano 2019 - 0 km,São Paulo - SP,"[Bancos de couro, Rodas de liga, Sensor de chu...",NaN
